# Calcul du nombre de prélèvements CVM non conforme par commune et par année

L'objectif de ce notebook est de partir de la liste de communes cog_communes, et pour chaque commune et chaque année, calculer le nombre de prélèvements non conformes pour le CVM.

Il y aura plusieurs aggrégations à faire :

- commune (inseecommune) peut avoir plusieurs UDIs (cdreseau) **ET** un UDI peut avoir plusieurs communes (inseecommune) 
- un prélèvement (referenceprel) peut être rattaché à plusieurs UDIs (cdreseau)
- un prélèvement (referenceprel) peut être composé de plusieurs paramètres (cdparametresiseeaux) ; mais dans le cas du CVM, il y a un seul paramètre selon la catégorisation de Pauline, donc c'est plus simple




In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 10

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

In [2]:
%%sql
SELECT * FROM int__resultats_udi_communes
LIMIT 5

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,valtraduite,limitequal,de_partition,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
00200179905,TCEYTCL,0.000,<=10 µg/L,2020,10.0,µg/L,hydrocarbures,002001064,02540,2020-05-19 11:48:00
00200179905,TCLEY,0.000,<=10 µg/L,2020,10.0,µg/L,hydrocarbures,002001064,02540,2020-05-19 11:48:00
00200179905,THM4,25.600,<=100 µg/L,2020,100.0,µg/L,sous produit désinfection,002001064,02540,2020-05-19 11:48:00
00200179905,TOL,0.000,None,2020,None,None,hydrocarbures,002001064,02540,2020-05-19 11:48:00
00200179905,XYLMEPA,0.000,None,2020,None,None,hydrocarbures,002001064,02540,2020-05-19 11:48:00


## Faire la liste complètes des communes (pour chaque catégorie et années)

In [3]:
%%sql --save list_communes
WITH
annees AS (
    SELECT unnest(generate_series(2020, 2024)) as annee
    )
   
SELECT DISTINCT
    annees.annee AS annee, 
    'cvm' AS categorie,
    inseecommune ,
 FROM
      int__lien_commune_cdreseau
 FULL OUTER JOIN
       annees
 ON
    int__lien_commune_cdreseau.de_partition = annees.annee

Running query in 'duckdb:///../../database/data.duckdb'

annee,categorie,inseecommune
2024,cvm,89009
2024,cvm,89015
2024,cvm,89051
2024,cvm,89073
2024,cvm,89238
2024,cvm,89363
2024,cvm,89370
2024,cvm,89408
2024,cvm,89474
2024,cvm,89480


In [4]:
%%sql --with list_communes
SELECT
    *
FROM
   list_communes
WHERE
    inseecommune = '07194'

Running query in 'duckdb:///../../database/data.duckdb'

annee,categorie,inseecommune
2023,cvm,07194
2020,cvm,07194
2021,cvm,07194
2022,cvm,07194
2024,cvm,07194


## Test cog pour le final

In [5]:
%%sql 
SELECT 
    *
FROM 
  cog_communes
WHERE
    TYPECOM = 'COM'

Running query in 'duckdb:///../../database/data.duckdb'

TYPECOM,COM,REG,DEP,CTCD,ARR,TNCC,NCC,NCCENR,LIBELLE,CAN,COMPARENT,de_partition,de_ingestion_date,de_dataset_datetime
COM,01001,84,01,01D,012,5,ABERGEMENT CLEMENCIAT,Abergement-Clémenciat,L'Abergement-Clémenciat,0108,None,2024,2025-03-16,20240220
COM,01002,84,01,01D,011,5,ABERGEMENT DE VAREY,Abergement-de-Varey,L'Abergement-de-Varey,0101,None,2024,2025-03-16,20240220
COM,01004,84,01,01D,011,1,AMBERIEU EN BUGEY,Ambérieu-en-Bugey,Ambérieu-en-Bugey,0101,None,2024,2025-03-16,20240220
COM,01005,84,01,01D,012,1,AMBERIEUX EN DOMBES,Ambérieux-en-Dombes,Ambérieux-en-Dombes,0122,None,2024,2025-03-16,20240220
COM,01006,84,01,01D,011,1,AMBLEON,Ambléon,Ambléon,0104,None,2024,2025-03-16,20240220
COM,01007,84,01,01D,011,1,AMBRONAY,Ambronay,Ambronay,0101,None,2024,2025-03-16,20240220
COM,01008,84,01,01D,011,1,AMBUTRIX,Ambutrix,Ambutrix,0101,None,2024,2025-03-16,20240220
COM,01009,84,01,01D,011,1,ANDERT ET CONDON,Andert-et-Condon,Andert-et-Condon,0104,None,2024,2025-03-16,20240220
COM,01010,84,01,01D,011,1,ANGLEFORT,Anglefort,Anglefort,0110,None,2024,2025-03-16,20240220
COM,01011,84,01,01D,014,1,APREMONT,Apremont,Apremont,0114,None,2024,2025-03-16,20240220


In [6]:
%%sql 
WITH
cog AS (
    SELECT 
      DEP AS code_departement,
      REG AS code_region,
      COM AS inseecommune,
      LIBELLE AS name_commune,
    FROM 
      cog_communes
    WHERE
        TYPECOM = 'COM'
)
    
SELECT
        inseecommune,
        COUNT(code_departement) AS nb_code_departement,
        COUNT(code_region) AS nb_code_region,
        COUNT(name_commune) AS nb_name_commune,
FROM
    cog
GROUP BY 
       inseecommune
HAVING 
    nb_code_departement > 1 OR nb_code_region >1 OR nb_name_commune>1

Running query in 'duckdb:///../../database/data.duckdb'

inseecommune,nb_code_departement,nb_code_region,nb_name_commune


## Les résultats par communes - Bilan annuel

###  SQL resultats_ALL_communes : Join avec list_communes_uid pour avoir la liste complète

###  Join avec cog pour avoir le nom des communes

### SQL complet

In [7]:
%%sql --save resultats_ALL_communes_name
WITH
annees AS (
    SELECT unnest(generate_series(2020, 2024)) AS annee
),

list_communes_uid AS (
    SELECT DISTINCT
        annees.annee,
        'cvm' AS categorie,
        com.inseecommune
    FROM
        int__lien_commune_cdreseau AS com
    FULL OUTER JOIN
        annees
        ON
            com.de_partition = annees.annee
),


    
communes_year AS (
    SELECT
        de_partition AS annee,
        categorie,
        inseecommune,
        sum(1) AS nb_analyses,
        sum(CASE
            WHEN
                limitequal_float IS NOT NULL AND valtraduite >= limitequal_float
                THEN 1
            ELSE 0
        END) AS nb_analyses_not_ok,
        sum(CASE
            WHEN
                limitequal_float IS NOT NULL AND valtraduite < limitequal_float
                THEN 1
            ELSE 0
        END) AS nb_analyses_ok
    FROM
        int__resultats_udi_communes
    WHERE 
        categorie = 'cvm'
    GROUP BY
        annee,
        categorie,
        inseecommune
),

int__resultats_communes AS (
    SELECT
        annee,
        inseecommune,
        categorie,
        coalesce(nb_analyses, 0) AS nb_analyses,
        coalesce(nb_analyses_not_ok, 0) AS nb_analyses_not_ok,
        coalesce(nb_analyses_ok, 0) AS nb_analyses_ok,
        CASE
            WHEN coalesce(nb_analyses, 0) = 0 THEN 'Pas recherché'
            WHEN
                coalesce(nb_analyses, 0) > 0 AND coalesce(nb_analyses_ok, 0) = 0
                THEN 'jamais quantifié'
            WHEN
                categorie = 'cvm'
                AND coalesce(nb_analyses, 0) > 0
                AND coalesce(nb_analyses_not_ok, 0) > 1
                THEN '> 0,5 µg/L'
            WHEN
                categorie = 'cvm'
                AND coalesce(nb_analyses, 0) > 0
                AND coalesce(nb_analyses_ok, 0) > 0
                THEN '<= 0,5 µg/L'
            ELSE 'Not CVM - other'
        END AS resultat
    FROM
        communes_year
),

int__resultats_all_communes AS (
    SELECT
        list_communes_uid.annee,
        list_communes_uid.inseecommune,
        list_communes_uid.categorie,
        coalesce(int__resultats_communes.nb_analyses, 0) AS nb_analyses,
        coalesce(int__resultats_communes.nb_analyses_not_ok, 0)
            AS nb_analyses_not_ok,
        coalesce(int__resultats_communes.nb_analyses_ok, 0) AS nb_analyses_ok,
        coalesce(int__resultats_communes.resultat, 'Pas recherché') AS resultat
    FROM
        list_communes_uid
    LEFT JOIN
        int__resultats_communes
        ON
            list_communes_uid.annee = int__resultats_communes.annee
            AND  list_communes_uid.categorie  =  int__resultats_communes.categorie
            AND list_communes_uid.inseecommune
            = int__resultats_communes.inseecommune
),

cog AS (
    SELECT
        dep AS code_departement,
        reg AS code_region,
        com AS inseecommune,
        libelle AS commune_nom
    FROM
        stg_communes__cog
    WHERE
        typecom = 'COM'
)

SELECT
    int__resultats_all_communes.inseecommune AS commune_code_insee,
    cog.commune_nom,
    'bilan annuel' AS periode,
    int__resultats_all_communes.categorie,
    int__resultats_all_communes.resultat,
    cast(concat(int__resultats_all_communes.annee, '-01-01') AS DATE)
        AS date_prvl
FROM
    int__resultats_all_communes
LEFT JOIN
    cog
    ON
        int__resultats_all_communes.inseecommune = cog.inseecommune


Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

commune_code_insee,commune_nom,periode,categorie,resultat,date_prvl
54018,Anoux,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54027,Atton,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54054,Bayon,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54071,Bezange-la-Grande,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54113,Champenoux,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54153,Dampvitoux,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54154,Deneuvre,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54164,Dommarie-Eulmont,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54175,Einvaux,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54183,Essey-la-Côte,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01


In [8]:
%%sql --with resultats_ALL_communes_name
SELECT 
    *
FROM
    resultats_ALL_communes_name
WHERE
     categorie = 'cvm'

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

commune_code_insee,commune_nom,periode,categorie,resultat,date_prvl
54059,Belleau,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54132,Clérey-sur-Brenon,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54170,Domptail-en-l'Air,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54200,Flirey,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54206,Fraimbois,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54218,Gélaucourt,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54431,Pont-à-Mousson,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54439,Pulnoy,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54519,Thiaville-sur-Meurthe,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54524,Thumeréville,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01


# Check

In [9]:
%%sql
SELECT
    de_partition as year,
    COUNT(DISTINCT inseecommune) AS nb_inseecommune
FROM 
 edc_communes
GROUP BY 
    1 
ORDER BY 
   1

Running query in 'duckdb:///../../database/data.duckdb'

year,nb_inseecommune
2020,34788
2021,34833
2022,34874
2023,34852
2024,34809


In [10]:
%%sql --with resultats_ALL_communes_name
 SELECT 
    date_prvl,
    COUNT(DISTINCT commune_code_insee) AS nb_inseecommune
 FROM
   resultats_ALL_communes_name
GROUP BY 
    1
ORDER BY 
   1

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

date_prvl,nb_inseecommune
2020-01-01,34788
2021-01-01,34833
2022-01-01,34874
2023-01-01,34852
2024-01-01,34809


#### commune_code_insee = '07194'

In [11]:
%%sql
 SELECT 
   de_partition, 
   COUNT(*)
 FROM
   int__resultats_udi_communes
WHERE
    inseecommune = '07194'
    AND categorie = 'cvm'
GROUP BY 
    de_partition
ORDER BY 
    de_partition

Running query in 'duckdb:///../../database/data.duckdb'

de_partition,count_star()
2020,3
2021,3
2022,11
2023,16
2024,10


In [12]:
%%sql --with resultats_ALL_communes_name 
 SELECT 
   *
 FROM
   resultats_ALL_communes_name
WHERE
    commune_code_insee = '07194'
    AND categorie = 'cvm'
ORDER BY 
    date_prvl

Running query in 'duckdb:///../../database/data.duckdb'

commune_code_insee,commune_nom,periode,categorie,resultat,date_prvl
07194,Rochessauve,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
07194,Rochessauve,bilan annuel,cvm,"<= 0,5 µg/L",2021-01-01
07194,Rochessauve,bilan annuel,cvm,"> 0,5 µg/L",2022-01-01
07194,Rochessauve,bilan annuel,cvm,"> 0,5 µg/L",2023-01-01
07194,Rochessauve,bilan annuel,cvm,"> 0,5 µg/L",2024-01-01


In [13]:
%%sql --with resultats_ALL_communes_name 
 SELECT 
   resultat, MIN(commune_code_insee)
 FROM
   resultats_ALL_communes_name
WHERE 
    categorie = 'cvm'
GROUP BY 1

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

resultat,min(commune_code_insee)
Pas recherché,01002
"> 0,5 µg/L",04018
"<= 0,5 µg/L",01001
jamais quantifié,08031


#### commune_code_insee = '01002'

In [15]:
%%sql 
 SELECT 
   *
 FROM
   int__resultats_udi_communes
WHERE
    inseecommune = '01002'
    AND categorie = 'cvm'
ORDER BY 
    de_partition

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdparametresiseeaux,valtraduite,limitequal,de_partition,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
00100130490,CLVYL,0.000,<=0.5 µg/L,2022,0.5,µg/L,cvm,001000369,01002,2022-04-26 09:56:00
00100139937,CLVYL,0.000,<=0.5 µg/L,2024,0.5,µg/L,cvm,001000369,01002,2024-03-26 15:10:00


In [16]:
%%sql --with resultats_ALL_communes_name 
 SELECT 
   *
 FROM
   resultats_ALL_communes_name
WHERE
    commune_code_insee = '01002'
    AND categorie = 'cvm'
ORDER BY 
    date_prvl

Running query in 'duckdb:///../../database/data.duckdb'

commune_code_insee,commune_nom,periode,categorie,resultat,date_prvl
01002,L'Abergement-de-Varey,bilan annuel,cvm,Pas recherché,2020-01-01
01002,L'Abergement-de-Varey,bilan annuel,cvm,Pas recherché,2021-01-01
01002,L'Abergement-de-Varey,bilan annuel,cvm,"<= 0,5 µg/L",2022-01-01
01002,L'Abergement-de-Varey,bilan annuel,cvm,Pas recherché,2023-01-01
01002,L'Abergement-de-Varey,bilan annuel,cvm,"<= 0,5 µg/L",2024-01-01


#### commune_code_insee = '08031'

In [19]:
%%sql 
 SELECT 
   *
 FROM
   int__resultats_udi_communes
WHERE
    inseecommune = '08031'
    AND categorie = 'cvm'
ORDER BY 
    de_partition

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdparametresiseeaux,valtraduite,limitequal,de_partition,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
00800092855,CLVYL,0.000,<=0.5 µg/L,2020,0.5,µg/L,cvm,008000543,08031,2020-12-17 12:45:00
00800092483,CLVYL,0.700,<=0.5 µg/L,2020,0.5,µg/L,cvm,008000543,08031,2020-11-17 12:44:00
00800094118,CLVYL,0.000,<=0.5 µg/L,2021,0.5,µg/L,cvm,008000543,08031,2021-05-27 09:19:00
00800102498,CLVYL,0.000,<=0.5 µg/L,2023,0.5,µg/L,cvm,008000543,08031,2023-09-22 14:14:00
00800105054,CLVYL,0.800,<=0.5 µg/L,2024,0.5,µg/L,cvm,008000543,08031,2024-06-10 12:21:00
00800106109,CLVYL,0.900,<=0.5 µg/L,2024,0.5,µg/L,cvm,008000543,08031,2024-09-24 12:24:00


In [18]:
%%sql --with resultats_ALL_communes_name 
 SELECT 
   *
 FROM
   resultats_ALL_communes_name
WHERE
    commune_code_insee = '08031'
    AND categorie = 'cvm'
ORDER BY 
    date_prvl

Running query in 'duckdb:///../../database/data.duckdb'

commune_code_insee,commune_nom,periode,categorie,resultat,date_prvl
08031,Aure,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
08031,Aure,bilan annuel,cvm,"<= 0,5 µg/L",2021-01-01
08031,Aure,bilan annuel,cvm,Pas recherché,2022-01-01
08031,Aure,bilan annuel,cvm,"<= 0,5 µg/L",2023-01-01
08031,Aure,bilan annuel,cvm,jamais quantifié,2024-01-01


# Final : ana__resultats_communes

In [23]:
%%sql
WITH
annees AS (
    SELECT unnest(generate_series(2020, 2024)) AS annee
),

list_communes_uid AS (
    SELECT DISTINCT
        annees.annee,
        'cvm' AS categorie,
        com.inseecommune
    FROM
        int__lien_commune_cdreseau AS com
    FULL OUTER JOIN
        annees
        ON
            com.de_partition = annees.annee
),

communes_year AS (
    SELECT
        de_partition AS annee,
        categorie,
        inseecommune,
        sum(1) AS nb_analyses,
        sum(CASE
            WHEN
                limitequal_float IS NOT NULL AND valtraduite >= limitequal_float
                THEN 1
            ELSE 0
        END) AS nb_analyses_not_ok,
        sum(CASE
            WHEN
                limitequal_float IS NOT NULL AND valtraduite < limitequal_float
                THEN 1
            ELSE 0
        END) AS nb_analyses_ok
    FROM
        int__resultats_udi_communes
    WHERE 
        categorie = 'cvm'
    GROUP BY
        annee,
        categorie,
        inseecommune
),

int__resultats_communes AS (
    SELECT
        annee,
        inseecommune,
        categorie,
        coalesce(nb_analyses, 0) AS nb_analyses,
        coalesce(nb_analyses_not_ok, 0) AS nb_analyses_not_ok,
        coalesce(nb_analyses_ok, 0) AS nb_analyses_ok,
        CASE
            WHEN coalesce(nb_analyses, 0) = 0 THEN 'Pas recherché'
            WHEN
                coalesce(nb_analyses, 0) > 0 AND coalesce(nb_analyses_ok, 0) = 0
                THEN 'jamais quantifié'
            WHEN
                categorie = 'cvm'
                AND coalesce(nb_analyses, 0) > 0
                AND coalesce(nb_analyses_not_ok, 0) > 1
                THEN '> 0,5 µg/L'
            WHEN
                categorie = 'cvm'
                AND coalesce(nb_analyses, 0) > 0
                AND coalesce(nb_analyses_ok, 0) > 0
                THEN '<= 0,5 µg/L'
            ELSE 'Not CVM - other'
        END AS resultat
    FROM
        communes_year
),

int__resultats_all_communes AS (
    SELECT
        list_communes_uid.annee,
        list_communes_uid.inseecommune,
        list_communes_uid.categorie,
        coalesce(int__resultats_communes.nb_analyses, 0) AS nb_analyses,
        coalesce(int__resultats_communes.nb_analyses_not_ok, 0)
            AS nb_analyses_not_ok,
        coalesce(int__resultats_communes.nb_analyses_ok, 0) AS nb_analyses_ok,
        coalesce(int__resultats_communes.resultat, 'Pas recherché') AS resultat
    FROM
        list_communes_uid
    LEFT JOIN
        int__resultats_communes
        ON
            list_communes_uid.annee = int__resultats_communes.annee
            AND  list_communes_uid.categorie  =  int__resultats_communes.categorie
            AND list_communes_uid.inseecommune
            = int__resultats_communes.inseecommune
),

cog AS (
    SELECT
        dep AS code_departement,
        reg AS code_region,
        com AS inseecommune,
        libelle AS commune_nom
    FROM
        stg_communes__cog
    WHERE
        typecom = 'COM'
)

SELECT
    int__resultats_all_communes.inseecommune AS commune_code_insee,
    cog.commune_nom,
    'bilan annuel' AS periode,
    int__resultats_all_communes.categorie,
    int__resultats_all_communes.resultat,
    cast(concat(int__resultats_all_communes.annee, '-01-01') AS DATE)
        AS date_prvl
FROM
    int__resultats_all_communes
LEFT JOIN
    cog
    ON
        int__resultats_all_communes.inseecommune = cog.inseecommune


Running query in 'duckdb:///../../database/data.duckdb'

commune_code_insee,commune_nom,periode,categorie,resultat,date_prvl
54018,Anoux,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54027,Atton,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54054,Bayon,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54071,Bezange-la-Grande,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54113,Champenoux,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54153,Dampvitoux,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54154,Deneuvre,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54164,Dommarie-Eulmont,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54175,Einvaux,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
54183,Essey-la-Côte,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01


In [24]:
%%sql
WITH
annees AS (
    SELECT unnest(generate_series(2020, 2024)) AS annee
),

list_communes_uid AS (
    SELECT DISTINCT
        annees.annee,
        'cvm' AS categorie,
        com.cdreseau
    FROM
        int__lien_commune_cdreseau AS com
    FULL OUTER JOIN
        annees
        ON
            com.de_partition = annees.annee
),

communes_year AS (
    SELECT
        de_partition AS annee,
        categorie,
        cdreseau,
        sum(1) AS nb_analyses,
        sum(CASE
            WHEN
                limitequal_float IS NOT NULL AND valtraduite >= limitequal_float
                THEN 1
            ELSE 0
        END) AS nb_analyses_not_ok,
        sum(CASE
            WHEN
                limitequal_float IS NOT NULL AND valtraduite < limitequal_float
                THEN 1
            ELSE 0
        END) AS nb_analyses_ok
    FROM
        int__resultats_udi_communes
    WHERE 
        categorie = 'cvm'
    GROUP BY
        annee,
        categorie,
        cdreseau
),

int__resultats_communes AS (
    SELECT
        annee,
        cdreseau,
        categorie,
        coalesce(nb_analyses, 0) AS nb_analyses,
        coalesce(nb_analyses_not_ok, 0) AS nb_analyses_not_ok,
        coalesce(nb_analyses_ok, 0) AS nb_analyses_ok,
        CASE
            WHEN coalesce(nb_analyses, 0) = 0 THEN 'Pas recherché'
            WHEN
                coalesce(nb_analyses, 0) > 0 AND coalesce(nb_analyses_ok, 0) = 0
                THEN 'jamais quantifié'
            WHEN
                categorie = 'cvm'
                AND coalesce(nb_analyses, 0) > 0
                AND coalesce(nb_analyses_not_ok, 0) > 1
                THEN '> 0,5 µg/L'
            WHEN
                categorie = 'cvm'
                AND coalesce(nb_analyses, 0) > 0
                AND coalesce(nb_analyses_ok, 0) > 0
                THEN '<= 0,5 µg/L'
            ELSE 'Not CVM - other'
        END AS resultat
    FROM
        communes_year
),

int__resultats_all_communes AS (
    SELECT
        list_communes_uid.annee,
        list_communes_uid.cdreseau,
        list_communes_uid.categorie,
        coalesce(int__resultats_communes.nb_analyses, 0) AS nb_analyses,
        coalesce(int__resultats_communes.nb_analyses_not_ok, 0)
            AS nb_analyses_not_ok,
        coalesce(int__resultats_communes.nb_analyses_ok, 0) AS nb_analyses_ok,
        coalesce(int__resultats_communes.resultat, 'Pas recherché') AS resultat
    FROM
        list_communes_uid
    LEFT JOIN
        int__resultats_communes
        ON
            list_communes_uid.annee = int__resultats_communes.annee
            AND  list_communes_uid.categorie  =  int__resultats_communes.categorie
            AND list_communes_uid.cdreseau
            = int__resultats_communes.cdreseau
)

SELECT
    int__resultats_all_communes.cdreseau AS cdreseau,
    'bilan annuel' AS periode,
    int__resultats_all_communes.categorie,
    int__resultats_all_communes.resultat,
    cast(concat(int__resultats_all_communes.annee, '-01-01') AS DATE)
        AS date_prvl
FROM
    int__resultats_all_communes


Running query in 'duckdb:///../../database/data.duckdb'

cdreseau,periode,categorie,resultat,date_prvl
054001027,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
054000688,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
054001884,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
054000346,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
055000082,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
055001252,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
055000955,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
055000565,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
055001060,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
056004531,bilan annuel,cvm,"<= 0,5 µg/L",2020-01-01
